In [1]:
import numpy as np
from csc.utils import *
from csc.atoms import *
from csc.dictionary import *
from itertools import product
import matplotlib.pyplot as plt

# Atom parameters
bmin = 0.005
bmax = 0.06
b_step = 0.005
b_values = np.arange(bmin, bmax, b_step)

ymin = 0.010
ymax = 0.036
y_step = 0.002
y_values = np.arange(ymin, ymax, y_step)

s_values = [1]

dictionary = ZSDictionary.from_values(b_values, y_values, s_values)
print('The dictionary has {} different atoms'.format(len(dictionary)))

The dictionary has 143 different atoms


In [2]:
batchSize = 200
input_signals_db = f'/home/gomes/synthetic-data/csc-signals-{batchSize}.json'
input_signals_db = f'C:\\Users\\gomes\\Documents\\ENS\\00_M1\\00_BORELLI\\01_WorkInProgress\\\synthetic-data\\csc-signals-{batchSize}.json'

In [3]:
lambda_10 = dictionary.alphaCSCCalibrationPerSNR(input_signals_db, snr=10, nb_signals=20, verbose=True)

    Calibration interval n°1 [1.00e-04, 1.00e+03] :
       0 - lmba = 1.00e-04 : nb_act = 28204
       1 - lmba = 5.99e-04 : nb_act = 28176
       2 - lmba = 3.59e-03 : nb_act = 27977
       3 - lmba = 2.15e-02 : nb_act = 26666
       4 - lmba = 1.29e-01 : nb_act = 17151
       5 - lmba = 7.74e-01 : nb_act = 4963
       6 - lmba = 4.64e+00 : nb_act = 0
       7 - lmba = 2.78e+01 : nb_act = 0
       8 - lmba = 1.67e+02 : nb_act = 0
       9 - lmba = 1.00e+03 : nb_act = 0
   >>> Last non-zero index : 5 for 4963 activations
    Calibration interval n°2 [7.74e-01, 4.64e+00] :
       0 - lmba = 7.74e-01 : nb_act = 4963
       1 - lmba = 9.45e-01 : nb_act = 3146
       2 - lmba = 1.15e+00 : nb_act = 1484
       3 - lmba = 1.41e+00 : nb_act = 184
       4 - lmba = 1.72e+00 : nb_act = 0
       5 - lmba = 2.09e+00 : nb_act = 0
       6 - lmba = 2.56e+00 : nb_act = 0
       7 - lmba = 3.12e+00 : nb_act = 0
       8 - lmba = 3.80e+00 : nb_act = 0
       9 - lmba = 4.64e+00 : nb_act = 0
   >>> Las

In [5]:
print(lambda_10)

2.094052927411524


In [4]:
with open(input_signals_db, 'r') as json_file:
    data = json.load(json_file)
    if data is None:
        raise ValueError("The input file is empty or does not contain any data.")

# Extract the signals from the DB
signals = data['signals']
snr_level = 15

signal_dict = next((signal for signal in signals if signal['snr'] == snr_level), None)
signal_id = signal_dict['id']

list_calibration = dictionary.alphaCSCCalibration(signal_dict)
for calibration in list_calibration:
    print(calibration)

(0.1, 0.000565771416460637, 16689)
(1.2000000000000002, 0.00316889840730295, 390)
(2.3000000000000003, 0.003523503276930701, 0)
(3.4000000000000004, 0.003523503276930701, 0)
(4.5, 0.003523503276930701, 0)
(5.6, 0.003523503276930701, 0)
(6.7, 0.003523503276930701, 0)
(7.800000000000001, 0.003523503276930701, 0)
(8.9, 0.003523503276930701, 0)
(10.0, 0.003523503276930701, 0)
